In [2]:
import os
import pandas as pd
import joblib

from sklearn.pipeline import make_pipeline
import numpy as np
from category_encoders import TargetEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
from category_encoders import MEstimateEncoder
from category_encoders import TargetEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from category_encoders import OneHotEncoder

In [3]:
sample_sub_df=pd.read_csv('..\input\sample_submission.csv',index_col=0)

In [4]:
class Dataset:
    def __init__(self,tgt_slide_d=28, train_period=28*5, feature_range=0):
        '''
        d1~d1913為訓練資料，共1913天，原始特徵提供。
        d1914~1941為驗證，共28天，應該是比賽結束後釋出。
        d1942~d1966為測試，共28天，隱藏起來的資料範圍。
        故訓練時應該用~D1885預測D1886~D1913
          驗證時應該用~D1913預測D1914~D1941
          測試時應該用~D1941預測D1942~D1966
          這樣驗證水準才會最近似於預測水準
        
        但若這樣會犧牲掉很多的資料，故若針對預測不同天的未來準備不同的資料會能使用到更近期的資料去訓練
        ex 預測下一天的資料可用~d1912預測~d1913
           預測下兩天的資料可用~d1911預測~d1913依此類推
        '''
        val_csv='..\input\sales_train_validation.csv'
        eval_csv='..\input\sales_train_evaluation.csv'
        sample_sub_csv='..\input\sample_submission.csv'   

        # val_df_id, val_df_d_sales = self.preprocess(val_csv) # 讀取資料
        # eval_df_id, eval_df_d_sales = self.preprocess(eval_csv) # 讀取資料

        val_data, val_id_org_col, val_id_onehot_col, val_d_sales_col = self.preprocess(val_csv) # 讀取資料
        eval_data, eval_id_org_col, eval_id_onehot_col, eval_d_sales_col = self.preprocess(eval_csv) # 讀取資料


        sample_sub_df=pd.read_csv(sample_sub_csv,index_col=0)

        # all_df_id = pd.concat([val_df_id,eval_df_id], axis=0)
        # all_df_d_sales = pd.concat([val_df_d_sales,eval_df_d_sales], axis=0)
        # sample_sub_df_id =all_df_id.loc(sample_sub_df.index)
        # sample_sub_df_sales = all_df_id.loc(sample_sub_df.index)
        
        #確認sliding window的測試資料會有多少個
        tgt_number = train_period//tgt_slide_d
        #計算出最早的tgt與最晚的tgt的時間差幾天
        real_train_period = (tgt_number-1)*tgt_slide_d
        #計算出最多一筆資料能使用多少長度的特徵，資料總長度減去實際的training基期的資料範圍，再減去使用的模型loop。
        max_feature_usage = 1913 - real_train_period - 28
        if feature_range > max_feature_usage:
            Exception('feature_range>max_feature_usage')
        elif feature_range==0:
            feature_range = max_feature_usage

        # self.df_train = val_df_d_sales.iloc[:,:-28]
        # self.df_val =  eval_df_d_sales.iloc[:,:-28]
        
        def get_data(tgt_d:int, df:pd.DataFrame, id_org_col:list, id_onehot_col:list, d_sales_col:list, is_eval=False):
            
            # print(id_onehot_col)
            if not is_eval:
                keys = df.index.to_list()
                values = [x+f'tgt_date_{tgt_d}' for x in df.index.to_list()]
                my_dict = dict(zip(keys, values))
                df = df.rename(index=my_dict)     
                df_sales = df[d_sales_col]                
                '''y'''
                if tgt_d==0:
                    y = df_sales.iloc[:,-28:]
                else:
                    y = df_sales.iloc[:,-28-tgt_d:-tgt_d]
                y.columns=[f'F{i+1}' for i in range(y.shape[1])]
                '''X'''                
                X = df_sales.iloc[:,-28-tgt_d-feature_range:-28-tgt_d]
                tgt_d = X.columns.to_list()[-1]
                
            else:
                '''y'''
                y = ''

                '''X''' 
                df_sales = df[d_sales_col]                 
                X = df_sales.iloc[:,-feature_range:]
                tgt_d = X.columns.to_list()[-1]                            
                
            
            #更改欄位名稱為前幾天
            
            X.columns=[f'Previous_Day_{i}' for i in range(X.shape[1],0,-1)]
            # print(X.head())
            # print(df.head())
            # df.reindex(columns=id_onehot_col)
            X=pd.concat([X,df[id_onehot_col]],axis=1)
            return X,y

        def get_x(is_train=False):
            pass

        self.X_train=pd.DataFrame()
        self.y_train=pd.DataFrame()
        for tgt_d in range(0,train_period,tgt_slide_d):
            X_tmp,y_tmp = get_data(tgt_d, val_data, val_id_org_col, val_id_onehot_col, val_d_sales_col)
            
            self.X_train=pd.concat([self.X_train,X_tmp],axis=0)
            self.y_train=pd.concat([self.y_train,y_tmp],axis=0)
            
            


        self.X_val,self.y_val = get_data(0,eval_data, eval_id_org_col, eval_id_onehot_col, eval_d_sales_col)
            
        # self.X_train = val_df_d_sales.iloc[:,:-28]
        # self.X_train.columns=[f'D_{i}' for i in range(self.X_train.shape[1])]
        # self.y_train = val_df_d_sales.iloc[:,-28:]
        
        # self.X_val = eval_df_d_sales.iloc[:,28:-28]
        # self.X_val.columns=[f'D_{i}' for i in range(self.X_val.shape[1])]
        # self.y_val = eval_df_d_sales.iloc[:,-28:]
        self.X_val_test, _ = get_data(0, val_data, val_id_org_col, val_id_onehot_col, val_d_sales_col,is_eval=True)
        self.X_eval_test, _ = get_data(0, eval_data, eval_id_org_col, eval_id_onehot_col, eval_d_sales_col,is_eval=True)
        
        self.X_test = pd.concat([self.X_val_test, self.X_eval_test],axis=0)
        # self.X_test =X_all.reindex(sample_sub_df.index.to_list())

        # self.X_train = self.concat(self.X_train, val_df_id)
        # self.X_val = self.concat(self.X_val, eval_df_id)

        # all_id=pd.concat([val_df_id,eval_df_id],axis=0)
        # self.X_test = self.concat(self.X_test, all_id.loc[sample_sub_df.index.to_list()])


        # self.X_train, self.X_local_val, self.y_train, self.y_local_val = train_test_split(self.x, self.y, test_size=0.2, random_state=42)

            # self.sales = pd.read_csv(submission_csv,index_col=0)
        # self._sales_processed = (
        #     sales
        #     .drop(["警示編號", "監控層級", "觸發說明"], axis=1)
        #     .assign(**sales.資料日期.dt.isocalendar(),
        #             month=sales.資料日期.dt.month,
        #             quarter=sales.資料日期.dt.quarter,
        #             hfy=sales.資料日期.dt.quarter.isin([3, 4])+1,
        #             bigMon=sales.資料日期.dt.month.isin([1, 3, 5, 7, 8, 10, 12]).astype(int))  # 將sales整理出更多的特徵資料
        # )
        # self.txn_c = txn[txn['tran_type'] == 'DEBIT'].drop("tran_type", axis=1)
        # self.txn_d = txn[txn['tran_type'] ==
        #                  'CREDIT'].drop("tran_type", axis=1)
        # self.txn_t = txn[txn['tran_type'] == 'TXN'].drop("tran_type", axis=1)

        # self._txn_processed = txn  # _get_data_from_sql整理出的txn資料
        # self._risk_processed = risk  # _get_data_from_sql整理出的risk資料
        # self._info_processed = info  # _get_data_from_sql整理出的info資料
        # self.ac = ac  # _get_data_from_sql從VW_NP_FSC_PARTY_ACCOUNT_BRIDGE整理出的客帳戶對照

        # self.train = self._get_data("train")  # VIP訓練資料
        # self.val = self._get_data("val")  # VIP驗證資料
        # self.test = self._get_data("test")  # VIP當天資料

        # self.seg_mapping = self._get_cust_seg_def()

    def preprocess(self,file_name):
        data = pd.read_csv(file_name,index_col=0) # 讀取資料
        id_org_col = data.iloc[:,:5].columns.to_list()
        d_sales_col = data.iloc[:,5:].columns.to_list()
        data_col_onehot = self._preprocess_obj_feature(data[id_org_col])
        id_onehot_col = data_col_onehot.columns.to_list()
        data=self.concat(data,data_col_onehot)
        return data, id_org_col, id_onehot_col, d_sales_col
    
    def concat(self, X_df, obj_df):

        return pd.concat([obj_df,X_df], axis=1)

    def _preprocess_obj_feature(self, obj_df):
        #onehot encoding feature
        self.encoder1 = OneHotEncoder()
        self.encoder1.fit(obj_df)
        obj_df = self.encoder1.transform(obj_df)

        return obj_df
    
    def preprocess_target(self, data, index):
        return data.loc[index].iloc[:,-28:]

In [4]:
# sample_sub_df=pd.read_csv('..\input\sample_submission.csv',index_col=0)
# sample_sub_df.index

In [5]:
# sample_sub_df.shape

In [6]:
# import pandas as pd

# # 创建示例 DataFrame
# data = {'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]}
# df = pd.DataFrame(data, index=['row1', 'row2', 'row3'])


In [7]:
# df.index.to_list()

In [8]:

# # 更改索引标签
# new_index_names = {'row1': 'new_row1', 'row2': 'new_row2', 'row3': 'new_row3'}
# new_df = df.rename(index=new_index_names)

# print(new_df)


In [5]:
dataset=Dataset()


In [6]:
dataset.y_train.columns[dataset.y_train.isna().any()].tolist()


[]

In [47]:
# dataset.X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30490 entries, HOBBIES_1_001_CA_1_validationtgt_date_0 to FOODS_3_827_WI_3_validationtgt_date_0
Columns: 4957 entries, Previous_Day_1885 to state_id_3
dtypes: float64(3072), int64(1885)
memory usage: 1.1+ GB


In [3]:
# test=pd.read_csv('..\input\sales_train_evaluation.csv',index_col=0)

In [27]:
# test.loc['HOBBIES_1_001_CA_1_evaluation']

item_id     HOBBIES_1_001
dept_id         HOBBIES_1
cat_id            HOBBIES
store_id             CA_1
state_id               CA
                ...      
d_1937                  0
d_1938                  3
d_1939                  3
d_1940                  0
d_1941                  1
Name: HOBBIES_1_001_CA_1_evaluation, Length: 1946, dtype: object

In [4]:
# test.head()

,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
id,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [67]:
# train_data.X_train

,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
id,,,,,,,,,,,,,,,,,,,,,
FOODS_3_164_CA_4_evaluation,0.983928,2.205954,1.806054,0.907511,1.436865,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
HOBBIES_1_198_TX_1_evaluation,1.037716,0.814663,0.669381,1.008855,1.165628,0,0,0,0,0,...,0,1,1,0,0,0,0,0,1,0
FOODS_1_084_CA_3_evaluation,1.118399,1.315278,1.806054,2.085602,1.436865,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
FOODS_2_180_CA_4_evaluation,1.252869,1.245477,1.806054,0.907511,1.436865,0,0,0,0,0,...,2,0,1,2,2,0,0,1,0,1
HOUSEHOLD_2_121_TX_1_evaluation,0.930140,0.347767,0.865138,1.008855,1.165628,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FOODS_3_137_WI_3_evaluation,1.091504,2.205954,1.806054,1.070187,1.159615,0,0,0,0,0,...,0,0,0,1,0,0,0,0,2,0
FOODS_3_117_CA_2_evaluation,1.629387,2.205954,1.806054,1.287635,1.436865,15,5,5,5,9,...,2,1,1,0,1,1,1,0,4,2
HOUSEHOLD_1_302_CA_1_evaluation,1.010822,1.365977,0.865138,1.466710,1.436865,0,3,0,1,0,...,3,1,1,0,0,0,0,0,2,0


In [12]:
class Models:
    def __init__(self, is_load=False, load_path='models'):
        '''


        '''
        
        package_dir = os.path.dirname(os.path.abspath(os.getcwd()))
        self.model_path = os.path.join(package_dir,load_path)
        os.makedirs(self.model_path, exist_ok=True)
        
        if is_load:
            self._load_models(load_path)
        else:
            self._set_28_models()

    # def _create_pipe(self, X_train):
    #     return make_pipeline(
    #         TargetEncoder(cols=X_train.columns[X_train.dtypes == 'object'],
    #                       handle_missing='return_nan'),
    #         SimpleImputer(missing_values=np.nan, strategy='mean'),
    #         XGBRegressor()
    #     )
    def _load_models(self,load_path):
        
        self.models = [joblib.load(os.path.join(self.model_path, fn))
                        for fn in sorted(os.listdir(self.model_path))]

    def _set_28_models(self):
        self.models=[]
        for i in range(28):
            self.models.append(XGBRegressor())

    def _train_28_models(self, X_train, X_local_val, y_train, y_local_val):
        for i in range(28):
            
            # from IPython import embed
            # embed()
            self.models[i].fit(X_train, y_train.iloc[:,i])
            self.models[i]
            y_pred = self.models[i].predict(X_local_val)
            # 計算均方根誤差
            mse = mean_squared_error(y_local_val.iloc[:,i], y_pred)
            rmse = np.sqrt(mse)
            print("mse:",mse,"     Root Mean Squared Error:", rmse)
            joblib.dump(self.models[i], os.path.join(
                self.model_path, f"model_day{str(i).zfill(2)}.pkl"), compress=3)
    def models_predict(self, X_val:pd.DataFrame, name='result_simple.csv'):

        output = X_val.iloc[:,0:1]
        for i in range(28):
            val_data_tmp_y = self.models[i].predict(X_val)
            output[f'F{i+1}'] = val_data_tmp_y

        output=output.iloc[:,1:]
        output.to_csv(name)


In [10]:
models=Models()
#519 17 5.429
#526 18 4.6
7*10

70

In [13]:
models._train_28_models(dataset.X_train,dataset.X_val,dataset.y_train,dataset.y_val)

mse: 3.990953347658384      Root Mean Squared Error: 1.997737056686486
mse: 3.3281554476279838      Root Mean Squared Error: 1.8243232848450912
mse: 3.5347837656356518      Root Mean Squared Error: 1.8801020625582143
mse: 3.5707828792865652      Root Mean Squared Error: 1.8896515232408766
mse: 4.272832285142021      Root Mean Squared Error: 2.067083037795536
mse: 6.065101025823118      Root Mean Squared Error: 2.4627425821273157
mse: 5.847492370514349      Root Mean Squared Error: 2.4181588803290714
mse: 4.916139024465091      Root Mean Squared Error: 2.2172367993665203
mse: 5.17350803931068      Root Mean Squared Error: 2.2745346863283222
mse: 4.735847066677701      Root Mean Squared Error: 2.1762001439843948
mse: 4.425248772990128      Root Mean Squared Error: 2.103627527151641
mse: 5.869327747565556      Root Mean Squared Error: 2.4226695498077233
mse: 6.401975257870526      Root Mean Squared Error: 2.5302124926318985
mse: 5.931856193455615      Root Mean Squared Error: 2.4355402262

In [14]:
output = dataset.X_test.iloc[:,0:1]
for i in range(28):
    val_data_tmp_y = models.models[i].predict(dataset.X_test)
    output[f'F{i+1}'] = val_data_tmp_y

output=output.iloc[:,1:]

In [ ]:
output.to_csv('result_simple.csv')

In [58]:
output = val_data.x.iloc[:,0:1]
for i in range(28):
    val_data_tmp_y = models.models[i].predict(val_data.x)
    output[f'F{i+1}'] = val_data_tmp_y

output=output.iloc[:,1:]

In [62]:
the_result=pd.concat([val_data.sales.iloc[:,0], output], axis=1)

In [63]:
the_result.to_csv('first_result.csv',index=False)

In [46]:
models.models[0].predict(train_data.X_local_val)

array([9.9999541e-01, 8.9999714e+00, 1.9999998e+00, ..., 9.9999541e-01,
       4.2390134e-06, 4.2390134e-06], dtype=float32)

In [49]:
train_data.y_local_val.iloc[:,0]

2404     1
7576     9
8731     2
4756     1
15509    0
        ..
8870     0
1423     0
27492    1
27020    0
21011    0
Name: d_1914, Length: 6098, dtype: int64

In [ ]:
mse = mean_squared_error(y_local_val[:,i], y_pred)

In [ ]:
models.models[0].predict(train_data.X_local_val)

In [14]:
models=Models(is_load=True)

In [15]:
models.models_predict(dataset.X_test, name='result2.csv')

In [42]:
train_data.y_train.iloc[:,0]


11535    0
12387    0
7792     0
11153    2
13413    0
        ..
29802    0
5390     0
860      1
15795    0
23654    1
Name: d_1914, Length: 24392, dtype: int64

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   A       3 non-null      int64 
 1   B       3 non-null      object
 2   C       3 non-null      bool  
dtypes: bool(1), int64(1), object(1)
memory usage: 179.0+ bytes


In [40]:
import pandas as pd

# 创建示例DataFrame
data = {'A': [1, 2, 3], 'B': ['foo', 'bar', 'baz'], 'C': [True, False, True]}
df = pd.DataFrame(data)

# 获取所有物件类型的列
object_columns = df.select_dtypes(exclude='object').columns

# 打印列名
print(df[object_columns])
# 这将打印出DataFrame中所有物件类型的列名。你可以根据需要使用这些列名进行进一步的操作，例如筛选特定的列或进行其他数据处理。








   A      C
0  1   True
1  2  False
2  3   True


In [33]:
# for x in train_data.sales.index:
#     print(x)

In [8]:


class model:
    def __init__(self, load=False):


        package_dir = os.path.dirname(os.path.abspath(__file__))
        self.model_path = os.path.join(
            package_dir, f'./models/')

        os.makedirs(self.model_path, exist_ok=True)

        if load:
            self.models = [joblib.load(os.path.join(self.model_path, fn))
                           for fn in sorted(os.listdir(self.model_path))]
        else:
            for fn in os.listdir(self.model_path):
                os.remove(os.path.join(self.model_path, fn))
            self.models = []
            self._train_inference = None
            self._train_27_models()

    def _create_pipe(self, X_train):
        return make_pipeline(
            TargetEncoder(cols=X_train.columns[X_train.dtypes == 'object'],
                          handle_missing='return_nan'),
            SimpleImputer(missing_values=np.nan, strategy='mean'),
            XGBRegressor()
        )
    def _train_27_models(self):
        for i in range(1, 28):
            train = self._create_label("train", i)
            x_train, y_train = train.drop(
                self.cols_dropped, axis=1), train.警示數量
            # from IPython import embed
            # embed()
            model = self._create_pipe(x_train)
            model.fit(x_train, y_train)
            joblib.dump(model, os.path.join(
                self.model_path, f"{datetime.now().date()}_w{str(i).zfill(2)}.pkl"), compress=3)
            self.models.append(model)



In [ ]:
model.get_booster().get_score()